# Train a scikit-learn SVM on the Iris dataset.

**Notebook Flow** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create and run a `Command` which executes a Python command
- Use a local file as an `input` to the Command

**Motivations** - This notebook explains how to setup and run a Command. The Command is a fundamental construct of Azure Machine Learning. It can be used to run a task on a specified compute (either local or on the cloud). The Command accepts `environment` and `compute` to setup required infrastructure. You can define a `command` to run on this infrastructure with `inputs`.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.identity import DefaultAzureCredential

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../../configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [2]:
# Enter details of your AML workspace
subscription_id = '6baab092-6683-4cc2-b4dd-7df2bb56af59'
resource_group = 'rg-treshenv-ws-a127'
workspace_name = 'ml-treshenv-ws-a127-svc-513c'

In [3]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)

## Connect to Compute for training

In [4]:
from azure.ai.ml.entities import AmlCompute

gpu_cluster = ml_client.compute.get("gpu-shared")
print(
         f" We will use cluster named {gpu_cluster.name} details: {gpu_cluster}"
    )

 We will use cluster named gpu-shared details: AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'gpu-shared', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/6baab092-6683-4cc2-b4dd-7df2bb56af59/resourceGroups/rg-treshenv-ws-a127/providers/Microsoft.MachineLearningServices/workspaces/ml-treshenv-ws-a127-svc-513c/computes/gpu-shared', 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f15bc56a680>, 'resource_id': None, 'location': 'canadacentral', 'size': 'STANDARD_NC6S_V3', 'min_instances': 1, 'max_instances': 1, 'idle_time_before_scale_down': 120.0, 'identity': <azure.ai.ml.entities._compute._identity.IdentityConfiguration object at 0x7f15bf6d7760>, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': None, 'tier': 'dedicated'})


# 2. Configure and run the Command
In this section we will configure and run a standalone job using the `command` class. The `command` class can be used to run standalone jobs and can also be used as a function inside pipelines.

## 2.1 Configure the Command
The `command` allows user to configure the following key aspects.
- `code` - This is the path where the code to run the command is located
- `command` - This is the command that needs to be run
- `inputs` - This is the dictionary of inputs using name value pairs to the command. The key is a name for the input within the context of the job and the value is the input value. Inputs can be referenced in the `command` using the `${{inputs.<input_name>}}` expression. To use files or folders as inputs, we can use the `Input` class. The `Input` class supports three parameters:
    - `type` - The type of input. This can be a `uri_file` or `uri_folder`. The default is `uri_folder`.         
    - `path` - The path to the file or folder. These can be local or remote files or folders. For remote files - http/https, wasb are supported. 
        - Azure ML `data`/`dataset` or `datastore` are of type `uri_folder`. To use `data`/`dataset` as input, you can use registered dataset in the workspace using the format '<data_name>:<version>'. For e.g Input(type='uri_folder', path='my_dataset:1')
    - `mode` - 	Mode of how the data should be delivered to the compute target. Allowed values are `ro_mount`, `rw_mount` and `download`. Default is `ro_mount`
- `environment` - This is the environment needed for the command to run. Curated or custom environments from the workspace can be used. Or a custom environment can be created and used as well. Check out the [environment](../../../../assets/environment/environment.ipynb) notebook for more examples.
- `compute` - The compute on which the command will run. In this example we are using a compute called `cpu-cluster` present in the workspace. You can replace it any other compute in the workspace. You can run it on the local machine by using `local` for the compute. This will run the command on the local machine and all the run details and output of the job will be uploaded to the Azure ML workspace.
- `distribution` - Distribution configuration for distributed training scenarios. Azure Machine Learning supports PyTorch, TensorFlow, and MPI-based distributed training. The allowed values are `PyTorch`, `TensorFlow` or `Mpi`.
- `display_name` - The display name of the Job
- `description` - The description of the experiment

In [5]:
# create the command
job = command(
    code="./src",  # local path where the code is stored
    command="python main.py --iris-csv ${{inputs.iris}} --C ${{inputs.C}} --kernel ${{inputs.kernel}} --coef0 ${{inputs.coef0}}",
    inputs={
        #"iris": Input(
        #    type="uri_file",
        #    path="https://azuremlexamples.blob.core.windows.net/datasets/iris.csv",
        #),

        # Use Registered Dataset
        "iris": Input(
            type="uri_file",
            path="iris-data:1",
        ),
        "C": 0.8,
        "kernel": "rbf",
        "coef0": 0.1,
    },
    environment="docker-sklearn@latest",           #"AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="gpu-shared",
    display_name="sklearn-iris-example",
    experiment_name="iris_model_mlflow_prediction",
    # description
)

## 2.2 Run the Command
Using the `MLClient` created earlier, we will now run this Command as a job in the workspace.

In [6]:
# submit the command
returned_job = ml_client.create_or_update(job)

In [7]:
# monitor the job

# stream the output and wait until the job is finished
ml_client.jobs.stream(returned_job.name)

# refresh the latest status of the job after streaming
returned_job = ml_client.jobs.get(name=returned_job.name)

RunId: happy_oregano_9l4yq8nx5v
Web View: https://ml.azure.com/runs/happy_oregano_9l4yq8nx5v?wsid=/subscriptions/6baab092-6683-4cc2-b4dd-7df2bb56af59/resourcegroups/rg-treshenv-ws-a127/workspaces/ml-treshenv-ws-a127-svc-513c

Streaming user_logs/std_log.txt

2022/10/27 14:08:31 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/miniconda/envs/amlenv/lib/python3.7/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
Registered model 'iris_svc_model' already exists. Creating a new version of this model...
2022/10/27 14:08:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris_svc_model, version 3
Created version '3' of model 'iris_svc_model'.
Registering the model via MLFlow

Execution Summary
RunId: happy_oregano_9l4yq8nx5v
Web View: https://ml.azure.com/runs/happy_oregano_9l4yq8nx5v?wsid=/subscriptions/6baab092-6683

# Next Steps
You can see further examples of running a job [here](../../../single-step/)